# Entropy
Entropy doesn't help.

In [81]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from joblib import Parallel, delayed

## Load Words

In [116]:
words_12k = pd.read_csv('words_12k.csv')
words_5k = pd.read_csv('words_5k.csv')
words_all = pd.read_table('data/en_words_1_5-5.txt', delimiter=' ', header=None, index_col=None,
                         names=['word_len', 'word_freq', 'n_articles']).reset_index()
words_all = words_all.rename(columns={'index': 'word'})

In [117]:
# Filter by english
alphabet = list('abcdefghijklmnopqrstuvwxyz')
words_all = words_all.loc[words_all.word.apply(lambda x: all([l in alphabet for l in x]))].reset_index(drop=True)

# At least 100
words_all = words_all.loc[words_all.word_freq.ge(100)]

## Game Logic

In [118]:
def wordle(input_word, solution):
    output = ''
    for i in range(5):
        if input_word[i] == solution[i]:
            output += 'G'
        elif input_word[i] in solution:
            output += 'Y'
        else:
            output += 'X'
    return output

In [119]:
def filter_wordset(input_word, feedback, wordset):
    newset = wordset.copy()
    for i in range(5):
        if feedback[i] == 'G':
            newset = newset.loc[newset.word.str[i] == input_word[i]]
        elif feedback[i] == 'Y':
            # newset = newset.loc[newset.word.str.contains(input_word[i])]
            newset = newset.loc[newset.word.str.contains(input_word[i]) & newset.word.apply(lambda x: x[i] != input_word[i])]
        else:
            newset = newset.loc[~newset.word.str.contains(input_word[i])]
    return newset

In [120]:
def compute_pos_letter_distribution(wordset):
    pos_scores = {}
    pos_scores[0] = wordset.word.str[0].value_counts().to_dict()
    pos_scores[1] = wordset.word.str[1].value_counts().to_dict()
    pos_scores[2] = wordset.word.str[2].value_counts().to_dict()
    pos_scores[3] = wordset.word.str[3].value_counts().to_dict()
    pos_scores[4] = wordset.word.str[4].value_counts().to_dict()
    
    return pos_scores

In [121]:
def compute_pos_score(letters, pos_scores):
    output = 0
    for i, letter in enumerate(letters):
        output += pos_scores[i][letter]
    return output

## Setup

In [123]:
pos_scores = compute_pos_letter_distribution(words_12k)
global_fl_scores = words_12k.word.apply(compute_pos_score, pos_scores=pos_scores)
global_fl_scores = pd.DataFrame({'word': words_12k.word, 'score': global_fl_scores}).sort_values('score', ascending=False)
entropy_table = pd.DataFrame(pos_scores.values()).T

In [124]:
from scipy.stats import entropy
def compute_entropy(word, total_words, entropy_table):
    output = 0
    for i, letter in enumerate(word):
        greens = entropy_table.loc[letter, i]
        yellows = np.sum(entropy_table.loc[letter, [pos for pos in range(5) if not pos == i]])
        greys = total_words - greens - yellows
    dist = np.array([greens, yellows, greys])
    dist = dist / np.sum(dist)
    return entropy(dist)

In [125]:
entropies = words_12k.word.apply(compute_entropy, total_words=words_12k.shape[0],
                                     entropy_table=pd.DataFrame(pos_scores.values()).T)

global_entropy = pd.DataFrame({'word': words_12k.word, 'score': entropies}).sort_values('score', ascending=False)

In [126]:
# Merge additional data
words_12k = words_12k.merge(words_all[['word', 'word_freq', 'n_articles']], how='left', left_on='word', right_on='word')
words_12k = words_12k.fillna(0)

words_5k = words_5k.merge(words_all[['word', 'word_freq', 'n_articles']], how='left', left_on='word', right_on='word')
words_5k = words_5k.fillna(0)

# Merge additional data
global_fl_scores = global_fl_scores.merge(words_all[['word', 'word_freq', 'n_articles']], how='left', left_on='word', right_on='word')
global_fl_scores = global_fl_scores.fillna(0)

global_entropy = global_entropy.merge(words_all[['word', 'word_freq', 'n_articles']], how='left', left_on='word', right_on='word')
global_entropy = global_entropy.fillna(0)

In [127]:
global_scores = global_fl_scores.merge(global_entropy[['word', 'score']] \
                                       .rename(columns={'score': 'entropy'}),
                                      how='left', on='word')

In [128]:
global_scores.sort_values(['entropy', 'score'], ascending=False).head(30)

,word,score,word_freq,n_articles,entropy
0,sores,10749,181.0,140.0,1.040935
1,sanes,10674,0.0,0.0,1.040935
2,sales,10561,26799.0,20964.0,1.040935
3,sones,10524,0.0,0.0,1.040935
4,soles,10411,425.0,335.0,1.040935
5,sates,10340,0.0,0.0,1.040935
6,seres,10313,0.0,0.0,1.040935
7,cares,10266,708.0,573.0,1.040935
8,bares,10249,0.0,0.0,1.040935
9,sames,10238,0.0,0.0,1.040935


## Double-word Strategy

In [130]:
def sim_wordle_double(input_word1, input_word2, solution):
    feedback = ''
    wordset = words_5k.copy()
    tested_words = []
    n_iter = 1
    
    input_word = input_word1

    while feedback != 'GGGGG':

        # Check solution
        feedback = wordle(input_word, solution)
        tested_words.append(input_word)
        # print(f'[{n_iter}] Input: {input_word} | Feedback: {feedback}')

        # Filter wordset
        wordset = filter_wordset(input_word, feedback, wordset)
        wordset = wordset.loc[~wordset.word.isin(tested_words)]

        # Compute letter distribution of updated wordset
        pos_scores = compute_pos_letter_distribution(wordset)
        
        # Compute entropy
        entropy_table = pd.DataFrame(pos_scores.values()).T.fillna(0)
        total_words = wordset.shape[0]
        
        # Obtain scores
        scores = wordset.word.apply(compute_entropy, total_words=total_words, entropy_table=entropy_table)
        
        scores = pd.DataFrame({'word': wordset.word, 'score': scores,
                              'word_freq': wordset.word_freq,
                              'n_articles': wordset.n_articles}).sort_values('score', ascending=False)
        scores = scores.sort_values(['score', 'n_articles'], ascending=False)
        # display(scores.head(10))

        # Set new input word
        if scores.shape[0] > 0:
            if len(tested_words) < 2:
                input_word = input_word2
            else:
                input_word = scores.word.iloc[0]
            n_iter += 1
    return n_iter

In [131]:
def double_word_strategy(input_word1, input_word2):
    results = Parallel(n_jobs=5, verbose=1)(delayed(sim_wordle_double)(input_word1, input_word2, s) for s in words_5k.word)
    performance = pd.Series(results)
    
    results = pd.DataFrame(performance.describe()).T
    results['word1'] = input_word1
    results['word2'] = input_word2
    display(results)
    
    return results

    # performance.plot.hist()
    # plt.title(f'{input_word1}, {input_word2}')
    # plt.show()

In [132]:
def find_second_pos(word, two_vowel=True):
    letters = list(word)
    candidates = global_fl_scores.loc[global_fl_scores.word.apply(lambda x: all([l not in letters for l in x]))]
    candidates = candidates.loc[candidates.word.apply(lambda x: len(x) == len(set(x)))]
    if two_vowel:
        candidates = candidates.loc[
            candidates.word.apply(lambda x: x.count('a') + x.count('e') + \
                                  x.count('i') + x.count('o') + x.count('u') >= 2)
        ]
    return candidates.head(20)

In [133]:
top_singles = global_fl_scores.loc[global_fl_scores.word.apply(lambda x: len(x) == len(set(x)))].word.head(10)
global_fl_scores.loc[global_fl_scores.word.apply(lambda x: len(x) == len(set(x)))].head(10)

,word,score,word_freq,n_articles
7,cares,10266,708.0,573.0
8,bares,10249,0.0,0.0
10,pares,10199,175.0,133.0
11,tares,10165,0.0,0.0
12,cores,10116,1270.0,827.0
14,bores,10099,213.0,181.0
18,pores,10049,681.0,533.0
19,mares,10044,1000.0,752.0
20,canes,10041,343.0,261.0
21,dares,10028,226.0,165.0


In [134]:
all_results = pd.DataFrame()
for input_word1 in top_singles:
    print('--------------------')
    print(input_word1)
    print('--------------------')
    all_word2 = find_second_pos(input_word1)
    print(all_word2.word.iloc[:5].tolist())

    for input_word2 in all_word2.word.iloc[:5]:
        print(f'[---- {input_word1}, {input_word2} ----]')
        res = double_word_strategy(input_word1, input_word2)
        all_results = all_results.append(res)
    print()
    print()
    print()

--------------------
cares
--------------------
['doily', 'pouty', 'poind', 'gouty', 'noily']
[---- cares, doily ----]


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    4.4s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:   16.0s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:   34.8s
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed:  1.0min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed:  1.7min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed:  2.4min
[Parallel(n_jobs=5)]: Done 2440 tasks      | elapsed:  3.4min
[Parallel(n_jobs=5)]: Done 3190 tasks      | elapsed:  4.5min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed:  5.9min
[Parallel(n_jobs=5)]: Done 4990 tasks      | elapsed:  7.5min
[Parallel(n_jobs=5)]: Done 5757 out of 5757 | elapsed:  8.8min finished


,count,mean,std,min,25%,50%,75%,max,word1,word2
0,5757.0,4.42001,1.349492,1.0,3.0,4.0,5.0,13.0,cares,doily


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


[---- cares, pouty ----]


...

,count,mean,std,min,25%,50%,75%,max,word1,word2
0,5757.0,4.374674,1.273891,1.0,3.0,4.0,5.0,12.0,cares,pouty


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


[---- cares, poind ----]


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    3.8s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:   17.5s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:   37.5s
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed:  1.1min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed:  1.7min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed:  2.6min
[Parallel(n_jobs=5)]: Done 2440 tasks      | elapsed:  3.5min
[Parallel(n_jobs=5)]: Done 3190 tasks      | elapsed:  4.8min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed:  6.1min


KeyboardInterrupt: 

In [39]:
all_results.to_csv('entropy_double_words.csv', index=False)

## Single-word Strategy

In [40]:
def sim_wordle_single(input_word, solution):
    feedback = ''
    wordset = words_5k.copy()
    tested_words = []
    n_iter = 1

    while feedback != 'GGGGG':

        # Check solution
        feedback = wordle(input_word, solution)
        tested_words.append(input_word)
        # print(f'[{n_iter}] Input: {input_word} | Feedback: {feedback}')

        # Filter wordset
        wordset = filter_wordset(input_word, feedback, wordset)
        wordset = wordset.loc[~wordset.word.isin(tested_words)]

        # Compute letter distribution of updated wordset
        pos_scores = compute_pos_letter_distribution(wordset)

        # Obtain scores
        scores = wordset.word.apply(compute_pos_score, pos_scores=pos_scores)
        scores = pd.DataFrame({'word': wordset.word, 'score': scores}).sort_values('score', ascending=False)
        # display(scores.head(10))

        # Set new input word
        if scores.shape[0] > 0:
            input_word = scores.word.iloc[0]
            n_iter += 1
    return n_iter

In [46]:
def single_word_strategy(input_word):
    results = Parallel(n_jobs=5, verbose=1)(delayed(sim_wordle_single)(input_word, s) for s in words_5k.word)
    performance = pd.Series(results)
    results = pd.DataFrame(performance.describe()).T
    results['word'] = input_word
    display(results)
    
    return results

    # performance.plot.hist()
    # plt.title(input_word)
    # plt.show()

In [47]:
top_singles = global_fl_scores.loc[global_fl_scores.word.apply(lambda x: len(x) == len(set(x)))].word.head(50)
print(top_singles.tolist())

['cares', 'bares', 'pares', 'tares', 'cores', 'bores', 'pores', 'mares', 'canes', 'dares', 'banes', 'tores', 'panes', 'fares', 'lares', 'bales', 'mores', 'cones', 'bones', 'pales', 'hares', 'gores', 'tales', 'pones', 'manes', 'fores', 'tones', 'wares', 'lores', 'coles', 'boles', 'fanes', 'poles', 'cates', 'males', 'lanes', 'nares', 'bates', 'dales', 'toles', 'rales', 'gales', 'pates', 'rones', 'vares', 'cames', 'kaies', 'kores', 'cotes', 'moles']


In [48]:
singles_results = pd.DataFrame()
for word in top_singles:
    print('--------------------')
    print(word)
    res = single_word_strategy(word)
    singles_results = singles_results.append(res)
    print('--------------------')
    print()
    print()
    print()

--------------------
cares


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  65 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done 665 tasks      | elapsed:    4.7s
[Parallel(n_jobs=5)]: Done 1665 tasks      | elapsed:   11.1s
[Parallel(n_jobs=5)]: Done 3065 tasks      | elapsed:   21.7s
[Parallel(n_jobs=5)]: Done 4865 tasks      | elapsed:   35.2s
[Parallel(n_jobs=5)]: Done 5757 out of 5757 | elapsed:   42.3s finished


,count,mean,std,min,25%,50%,75%,max,word
0,5757.0,4.42001,1.344849,1.0,4.0,4.0,5.0,13.0,cares


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


--------------------



--------------------
bares


...

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


--------------------



--------------------
moles


[Parallel(n_jobs=5)]: Done 110 tasks      | elapsed:    0.8s
[Parallel(n_jobs=5)]: Done 710 tasks      | elapsed:    5.9s
[Parallel(n_jobs=5)]: Done 1710 tasks      | elapsed:   13.2s
[Parallel(n_jobs=5)]: Done 3110 tasks      | elapsed:   23.5s
[Parallel(n_jobs=5)]: Done 4910 tasks      | elapsed:   36.9s
[Parallel(n_jobs=5)]: Done 5757 out of 5757 | elapsed:   43.2s finished


,count,mean,std,min,25%,50%,75%,max,word
0,5757.0,4.373458,1.280708,1.0,4.0,4.0,5.0,13.0,moles


--------------------





In [33]:
all_results.sort_values('mean')

,count,mean,std,min,25%,50%,75%,max,word1,word2
0,5757.0,4.131839,1.118184,1.0,3.0,4.0,5.0,11.0,pares,doilt
0,5757.0,4.157721,1.144281,1.0,3.0,4.0,5.0,11.0,dares,point
0,5757.0,4.161195,1.134188,1.0,3.0,4.0,5.0,11.0,tares,poind
0,5757.0,4.162585,1.158392,1.0,3.0,4.0,5.0,11.0,cores,paint
0,5757.0,4.209484,1.165684,1.0,3.0,4.0,5.0,12.0,tares,could
0,5757.0,4.220254,1.157710,1.0,3.0,4.0,5.0,11.0,pores,canid
0,5757.0,4.222512,1.159752,1.0,3.0,4.0,5.0,11.0,cares,poind
0,5757.0,4.228591,1.147040,1.0,3.0,4.0,5.0,12.0,pares,could
0,5757.0,4.230676,1.185069,1.0,3.0,4.0,5.0,11.0,tares,piony
0,5757.0,4.230849,1.181585,1.0,3.0,4.0,5.0,12.0,pores,laity


In [49]:
singles_results.sort_values('mean')

,count,mean,std,min,25%,50%,75%,max,word
0,5757.0,4.317700,1.278730,2.0,3.0,4.0,5.0,13.0,toles
0,5757.0,4.338371,1.304617,1.0,3.0,4.0,5.0,13.0,tales
0,5757.0,4.352614,1.284847,1.0,4.0,4.0,5.0,12.0,pores
0,5757.0,4.352788,1.287299,1.0,4.0,4.0,5.0,13.0,pales
0,5757.0,4.355046,1.318619,2.0,3.0,4.0,5.0,12.0,tores
0,5757.0,4.360778,1.322919,1.0,3.0,4.0,5.0,12.0,tares
0,5757.0,4.362689,1.283529,1.0,4.0,4.0,5.0,13.0,males
0,5757.0,4.369290,1.287056,1.0,4.0,4.0,5.0,13.0,poles
0,5757.0,4.373458,1.280708,1.0,4.0,4.0,5.0,13.0,moles
0,5757.0,4.379538,1.338329,1.0,4.0,4.0,5.0,13.0,tones


In [50]:
singles_results.to_csv('single_words.csv', index=False)

## Doubles Search

In [80]:
doubles_search = pd.DataFrame()
for input_word1 in ['toles', 'tales', 'pores', 'pales', 'tores', 'tares', 'pares']:
    print('--------------------')
    print(input_word1)
    print('--------------------')
    all_word2 = find_second_pos(input_word1)
    print(all_word2.word.iloc[:10].tolist())

    for input_word2 in all_word2.word.iloc[:10]:
        print(f'[---- {input_word1}, {input_word2} ----]')
        res = double_word_strategy(input_word1, input_word2)
        doubles_search = all_results.append(res)
    print()
    print()
    print()

--------------------
toles
--------------------
['rainy', 'dairy', 'fairy', 'marid', 'canid', 'hairy', 'caird', 'vairy', 'ranid', 'cairn']
[---- toles, rainy ----]


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    3.0s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:   14.7s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:   33.0s


KeyboardInterrupt: 